In [1]:
import os, sys
import re
import datetime
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

plt.rcParams.update({'font.size': 14})

In [2]:
testname_re = re.compile("test_(\d+)-(\d+).py")

In [3]:
testsuiteNameCache = {}
def getTestsuiteNameFromFile(file):
    global testsuiteNameCache
    file = file.replace("/home/bankde/", "/Users/bankde/Desktop/project/final_project/Volpy_main/")
    s = file.replace("-","/").split("/")
    cur_test = s[-3]+"/"+s[-2]
    with open(file, "r") as f:
        for line in f:
            if line.startswith("###### Description ######"):
                testsuite_name = (f.readline()).strip("#\ \n")
                testsuiteNameCache[cur_test] = testsuite_name
                return testsuite_name
            
def getTestsuiteName(file):
    global testsuiteNameCache
    file = file.replace("/home/bankde/", "/Users/bankde/Desktop/project/final_project/Volpy_main/")
    s = cur_file.replace("-","/").split("/")
    cur_test = s[-3]+"/"+s[-2]
    return testsuiteNameCache[cur_test]

In [4]:
result_file = "result_gcloud.json"
with open(result_file, "r") as f:
    raw_results = json.load(f)

In [5]:
args = type('', (), {})
args.data = "size"

In [6]:
result_dict = {}
for cur_pickle in raw_results["pickleSuites"]:
    result_dict[cur_pickle] = {}
    # First loop to init the dict with testsuite name
    for cur_file in raw_results[cur_pickle]:
        if cur_file.endswith("-1.py"):
            cur_testname = getTestsuiteNameFromFile(cur_file)
            if cur_testname in result_dict[cur_pickle]:
                raise Exception("Impossible flow") # Only one init file should exist.
            result_dict[cur_pickle][cur_testname] = [{} for i in range(2)] # 0/1 for init/test
    # Add the result to our result_dict
    for cur_file in raw_results[cur_pickle]:
        # Get args.data (default: "result") from raw_result
        results = {k: raw_results[cur_pickle][cur_file][k][args.data] if args.data in raw_results[cur_pickle][cur_file][k] else "NA" for k in raw_results[cur_pickle][cur_file]}
        if cur_file.endswith("-1.py"):
            result_dict[cur_pickle][getTestsuiteName(cur_file)][0] = results
        else:
            # There may be several test file
            result_dict[cur_pickle][getTestsuiteName(cur_file)][1].update(results)

In [7]:
# Beauty print with pandas
all_results = {}
for testsuite in result_dict[list(result_dict.keys())[0]]:
    df_all_pickles = []
    for cur_pickle in result_dict:
        df = pd.DataFrame(data=result_dict[cur_pickle][testsuite])
        df = df.fillna(' ').T
        df = df.rename(columns={0: "Init(%s)" % (cur_pickle), 1: "Test(%s)" % (cur_pickle)})
        df_all_pickles.append(df)
    result = pd.concat(df_all_pickles, axis=1)
    all_results[testsuite] = result
#     print(result)
#     result.plot(kind="box")
#     summary_fh.write("===== Testsuite: %s =====\n" % (testsuite))
#     summary_fh.write(str(result))
#     summary_fh.write("\n")

In [8]:
# For exec
exec_testsuiteNameCache = {}
def exec_getTestsuiteNameFromFile(file):
    global exec_testsuiteNameCache
    file = file.replace("/home/bankde/", "/Users/bankde/Desktop/project/final_project/Volpy_main/")
    with open(file, "r") as f:
        for line in f:
            if line.startswith("###### Description ######"):
                testsuite_name = (f.readline()).strip("#\ \n")
                exec_testsuiteNameCache[file] = testsuite_name
                return testsuite_name
            
def exec_getTestsuiteName(file):
    global exec_testsuiteNameCache
    file = file.replace("/home/bankde/", "/Users/bankde/Desktop/project/final_project/Volpy_main/")
    return exec_testsuiteNameCache[file]

In [16]:
# Load from exec results
exec_result_file = "../exec/result_gcloud.json"
with open(exec_result_file, "r") as f:
    exec_raw_results = json.load(f)
cur_pickle = "exec"
exec_result_dict = {}
exec_result_dict[cur_pickle] = {}
# First loop to init the dict with testsuite name
for cur_file in exec_raw_results[cur_pickle]:
    cur_testname = exec_getTestsuiteNameFromFile(cur_file)
    exec_result_dict[cur_pickle][cur_testname] = [{} for i in range(2)] # 0/1 for init/test
# Add the result to our result_dict
for cur_file in exec_raw_results[cur_pickle]:
    # Get args.data (default: "result") from raw_result
    results = {k: exec_raw_results[cur_pickle][cur_file][k][args.data] if args.data in exec_raw_results[cur_pickle][cur_file][k] else "NA" for k in exec_raw_results[cur_pickle][cur_file]}
    # For exec, there should be only Test, no init.
    exec_result_dict[cur_pickle][exec_getTestsuiteName(cur_file)][1] = results
    
exec_all_results = {}
for testsuite in exec_result_dict[list(exec_result_dict.keys())[0]]:
    df_all_pickles = []
    for cur_pickle in exec_result_dict:
        df = pd.DataFrame(data=exec_result_dict[cur_pickle][testsuite])
        df = df.fillna(' ').T
        df = df.rename(columns={0: "Init(%s)" % (cur_pickle), 1: "Test(%s)" % (cur_pickle)})
        df_all_pickles.append(df)
    result = pd.concat(df_all_pickles, axis=1)
    exec_all_results[testsuite] = result

In [10]:
# List all testsuite
for ts in all_results:
    print(ts)
    for test in all_results[ts].index:
        print("    %s" % test)

Closure
    test_func_closure
Recursive_function
    test_func_recursive
    test_func_recursive_double
Function_scope
    test_func_shared_scope
Functions
    test_func_chain
    test_func_simple
    test_func_simple_with_default_arg
Modules
    test_module_alias
    test_module_closure
    test_module_simple
Classes
    test_class_nested
    test_class_object
    test_class_simple
Inherit_class
    test_class_inherit
    test_class_inherit_instance
Distributed_sort
    test_distributed_sort


In [23]:
# List all testsuite
for testsuite in all_results:
    print(testsuite)
    for test in all_results[testsuite].index:
        print("  %s" % (test))
        print("    Init(codepickle): %d" % all_results[testsuite]["Init(codepickle)"][test])
        print("    Init(cloudpickle): %d" % all_results[testsuite]["Init(cloudpickle)"][test])
        print("    Init(exec): %s" % exec_all_results[testsuite]["Test(exec)"][test])

Closure
  test_func_closure
    Init(codepickle): 532
    Init(cloudpickle): 605
    Init(exec): 78.0
Recursive_function
  test_func_recursive
    Init(codepickle): 523
    Init(cloudpickle): 558
    Init(exec): 73.0
  test_func_recursive_double
    Init(codepickle): 692
    Init(cloudpickle): 708
    Init(exec): 157.0
Function_scope
  test_func_shared_scope
    Init(codepickle): 630
    Init(cloudpickle): 661
    Init(exec): 111.0
Functions
  test_func_chain
    Init(codepickle): 599
    Init(cloudpickle): 789
    Init(exec): 88.0
  test_func_simple
    Init(codepickle): 473
    Init(cloudpickle): 522
    Init(exec): 34.0
  test_func_simple_with_default_arg
    Init(codepickle): 500
    Init(cloudpickle): 536
    Init(exec): 47.0
Modules
  test_module_alias
    Init(codepickle): 564
    Init(cloudpickle): 615
    Init(exec): 120.0
  test_module_closure
    Init(codepickle): 654
    Init(cloudpickle): 732
    Init(exec): 179.0
  test_module_simple
    Init(codepickle): 557
    Init(clo